In [2]:
!pip install gdown

In [3]:
!pip install wandb

In [4]:
#wandb
from kaggle_secrets import UserSecretsClient
import wandb
user_secrets = UserSecretsClient()

my_secret = user_secrets.get_secret("bert-rcnn2") 

wandb.login(key=my_secret)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: storknguyen-286 (storknguyen-286-university-system-of-georgia) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [5]:
!gdown 18ZT8T7cUHvHNqoe_aZiDslR07lodEOK_
!unzip sentiment.zip
!rm sentiment.zip 
!ls

Downloading...
From (original): https://drive.google.com/uc?id=18ZT8T7cUHvHNqoe_aZiDslR07lodEOK_
From (redirected): https://drive.google.com/uc?id=18ZT8T7cUHvHNqoe_aZiDslR07lodEOK_&confirm=t&uuid=6e8c2982-d896-4d24-9bad-c737f29fcb09
To: /kaggle/working/sentiment.zip
100%|█████████████████████████████████████████| 672M/672M [00:06<00:00, 101MB/s]
Archive:  sentiment.zip
  inflating: multi_cased_L-12_H-768_A-12/vocab.txt  
  inflating: multi_cased_L-12_H-768_A-12/bert_config.json  
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: NTC_SV/NTC_SV_test.csv  
  inflating: NTC_SV/NTC_SV_train.csv  
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
multi_cased_L-12_H-768_A-12  NTC_SV


In [6]:
!pip install transformers
!pip install torch
!pip install scikit-learn
!pip install numpy
!pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.8 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.9.90
    Uninstalling nvidia-curand-cu12-1

In [7]:
# BERT + RCNN với PyTorch cho bài toán phân loại văn bản FOODY
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
from tqdm import tqdm
import wandb

2025-04-24 16:55:46.582420: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745513746.775065      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745513746.826221      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [8]:
# 1 Load dữ liệu
train_path = "/kaggle/working/NTC_SV/NTC_SV_train.csv"
test_path = "/kaggle/working/NTC_SV/NTC_SV_test.csv"
df = pd.read_csv(train_path).dropna()
df_test = pd.read_csv(test_path).dropna()

In [9]:
class FoodyDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=256):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'label': torch.tensor(label, dtype=torch.long)
        }

In [10]:
# 4. Mô hình BERT + RCNN
class BERT_RCNN(nn.Module):
    def __init__(self, bert_model_name='bert-base-multilingual-cased', hidden_size=768, rnn_hidden=128, num_classes=3):
        super(BERT_RCNN, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.bi_gru = nn.GRU(hidden_size, rnn_hidden, bidirectional=True, batch_first=True)
        self.conv1d = nn.Conv1d(2 * rnn_hidden, 128, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(128, num_classes)

    def forward(self, input_ids, attention_mask):
        with torch.no_grad():
            outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state
        rnn_out, _ = self.bi_gru(sequence_output)
        rnn_out = rnn_out.permute(0, 2, 1)
        conv_out = self.relu(self.conv1d(rnn_out))
        pooled = torch.max(conv_out, dim=2)[0]
        return self.fc(pooled)

In [ ]:
# 5. Khởi tạo tokenizer, dataset, dataloader
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
train_dataset = FoodyDataset(df['review'].tolist(), df['label'].tolist(), tokenizer)
test_dataset = FoodyDataset(df_test['review'].tolist(), df_test['label'].tolist(), tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [12]:
# 6. Khởi tạo mô hình, loss, optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BERT_RCNN(num_classes=len(set(df['label']))).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=2e-5)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [13]:
print(len(set(df['label'])))

2


In [12]:
print(device)

cuda


In [13]:
# 7. Huấn luyện
num_epochs = 10
save_every = 2
save_dir = "checkpoints"
os.makedirs(save_dir, exist_ok=True)

In [14]:
import wandb
import os
from tqdm import tqdm

# Khởi tạo W&B (chạy 1 lần đầu tiên sẽ yêu cầu đăng nhập)
wandb.init(project="my-project", config={
    "epochs": num_epochs,
    "batch_size": train_loader.batch_size,
    "learning_rate": optimizer.param_groups[0]['lr'],
    "model": "BERT + RCNN",
})

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct, total = 0, 0
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")

    for batch in loop:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

        loop.set_postfix(loss=loss.item())

        # 👉 Log loss từng batch (tuỳ chọn)
        wandb.log({"batch_loss": loss.item()})

    # 👉 Log loss & accuracy mỗi epoch
    epoch_loss = total_loss / len(train_loader)
    epoch_acc = correct / total
    wandb.log({
        "epoch": epoch + 1,
        "loss": epoch_loss,
        "accuracy": epoch_acc
    })

    print(f"\n✅ Epoch {epoch+1} — Loss: {epoch_loss:.4f} — Accuracy: {epoch_acc:.2%}")

    # 👉 Lưu mô hình định kỳ
    if (epoch + 1) % save_every == 0:
        ckpt_path = os.path.join(save_dir, f"bert_rcnn_epoch{epoch+1}.pth")
        torch.save(model.state_dict(), ckpt_path)
        print(f"💾 Đã lưu mô hình tại: {ckpt_path}")

        # 👉 Log checkpoint vào W&B
        wandb.save(ckpt_path)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch 1/10: 100%|██████████| 2548/2548 [06:15<00:00,  6.78it/s, loss=0.414] 



✅ Epoch 1 — Loss: 0.4785 — Accuracy: 78.90%


Epoch 2/10: 100%|██████████| 2548/2548 [06:13<00:00,  6.82it/s, loss=0.302] 



✅ Epoch 2 — Loss: 0.3872 — Accuracy: 83.37%
💾 Đã lưu mô hình tại: checkpoints/bert_rcnn_epoch2.pth


...


✅ Epoch 8 — Loss: 0.3099 — Accuracy: 87.08%
💾 Đã lưu mô hình tại: checkpoints/bert_rcnn_epoch8.pth


Epoch 9/10: 100%|██████████| 2548/2548 [06:15<00:00,  6.79it/s, loss=0.152] 



✅ Epoch 9 — Loss: 0.3045 — Accuracy: 87.30%


Epoch 10/10: 100%|██████████| 2548/2548 [06:14<00:00,  6.80it/s, loss=0.575] 



✅ Epoch 10 — Loss: 0.2971 — Accuracy: 87.73%
💾 Đã lưu mô hình tại: checkpoints/bert_rcnn_epoch10.pth


In [24]:
# Lưu tokenizer
save_dir_token_model = '/kaggle/working/token'
tokenizer.save_pretrained(save_dir_token_model)
print(f"Đã lưu tokenizer tại: {save_dir_token_model}")


Đã lưu tokenizer tại: /kaggle/working/token


In [26]:
!zip -r tokenizer_bert_rcnn.zip /kaggle/working/token

  adding: kaggle/working/token/ (stored 0%)
  adding: kaggle/working/token/tokenizer_config.json (deflated 75%)
  adding: kaggle/working/token/vocab.txt (deflated 45%)
  adding: kaggle/working/token/special_tokens_map.json (deflated 42%)


In [21]:
!pip install safetensors

In [23]:
from safetensors.torch import save_file
import os

# Lưu state_dict của model
state_dict = model.state_dict()

# Đường dẫn để lưu
save_path = os.path.join("/kaggle/working/", "model.safetensors")
save_file(state_dict, save_path)

print(f"✅ Đã lưu trọng số safetensors tại: {save_path}")


✅ Đã lưu trọng số safetensors tại: /kaggle/working/model.safetensors


In [19]:
# 👉 Đánh giá mô hình trên tập test
model.eval()
test_correct, test_total = 0, 0

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        
        outputs = model(input_ids, attention_mask)
        preds = outputs.argmax(dim=1)
        test_correct += (preds == labels).sum().item()
        test_total += labels.size(0)

test_acc = test_correct / test_total
print(f"\n📊 Độ chính xác trên tập test: {test_acc:.2%}")

# 👉 Log độ chính xác test vào W&B
wandb.log({"test_accuracy": test_acc})


📊 Độ chính xác trên tập test: 88.58%
